# Electricity Meter Reading Pipeline - Stage 3: Digit Recognition

This document describes the process for training a digit recognition model as part of an electricity meter reading pipeline.

## Preprocessing

The following steps are involved in preprocessing the images of the electricity meter counter:

1. **Input:** Take the output of Stage 2, which is an image of the counter area cropped from the original image.
2. **Straighten:** Use horizontal line detection (e.g., Hough Line Transform) to estimate the rotation angle and straighten the image. This ensures proper digit alignment.
3. **Convert to Binary:**
   - Convert the image to grayscale.
   - Apply binary thresholding to create a black and white (B/W) image.
   - Invert the binary image to make the digits white and the background black.
4. **Remove Borders:** Use horizontal line detection to identify and remove any remaining borders above and below the digits. This can be achieved using techniques like inpainting or morphological operations.
5. **Isolate Digits:** Apply edge detection (e.g., Canny edge detector) to identify the boundaries of the digits.
6. **Filter Contours:** Filter the resulting contours (detected digit boundaries) based on criteria such as:
   - Height > Width (digits are usually taller than they are wide)
   - Area (height * width) > threshold (to eliminate small noise or artifacts)
   - Other conditions as needed (e.g., aspect ratio, solidity)
7. **Extract and Save:** Extract each segmented digit and save it as a separate image file.

In [ ]:
# Import the required libraries

import cv2
import os
import numpy as np
import glob
from predict_helpers import *


In [ ]:
# Define the source images:
# Note Shoulp be the "crop" output from Stage 2 Detection

image_path = "/Users/yonz/Workspace/meterreader2/meterreader_YOLO/counter-1/crops/counter/IMG_69*.jpg"
images = glob.glob(image_path)

if len(images) > 0:
    print(f"{len(images)} files matching {image_path} found")
else:
    print(f"No images found matching {image_path}")


In [ ]:

# iterate over all the images
for img_path in images:
    # 1 load the image
    image = load_image(img_path)
    plot_image(image)

    # 2. Straighten
    #       Attemt to rotate the image so that the horizontal lines are straight,
    rotation_angle = determine_rotation_angle(image, horizontal_threshold=0.1)          
    image_rotated = rotate_image(image, rotation_angle)
    print(f"Counter Image - Rotated by {rotation_angle:.3f} degrees")
    plot_image(image_rotated, title=f"Counter Image - Rotated by {rotation_angle:.3f} degrees", bgr=False)

    # 3. Convert to Binary / Inverse
    image_binary = convert_to_binary(image_rotated, bgr=False, invert=True)
    print(f"Binary Image - Created: Shape: {image_binary.shape}. Dtype: {image_binary.dtype}")
    plot_image(image_binary, title=f"Binary Image -", bgr=False)

    # 4 Remove Borders
    # Develop some magic here.....